This Notebook shows introduces the basic concepts of RDDs and operations on them visually, by showing the contents of the RDDs as a table.

### Display RDD
The following helper functions displays the current contents of an RDD (partition-by-partition). This is best used for small RDDs with manageable number of partitions.

In [1]:
class DisplayRDD:
    def __init__(self, rdd):
        self.rdd = rdd

    def _repr_html_(self):                                  
        x = self.rdd.mapPartitionsWithIndex(lambda i, x: [(i, [y for y in x])])
        l = x.collect()
        s = "<table><tr>{}</tr><tr><td>".format("".join(["<th>Partition {}".format(str(j)) for (j, r) in l]))
        s += '</td><td valign="bottom" halignt="left">'.join(["<ul><li>{}</ul>".format("<li>".join([str(rr) for rr in r])) for (j, r) in l])
        s += "</td></table>"
        return s

### Basics 1
Lets start with some basic operations using a small RDD to visualize what's going on. We will create a RDD of Strings, using the `states.txt` file which contains a list of the state names.

The notebook has already initialized a SparkContext, and we can refer to it as `sc`.

We will use `sc.textFile` to create this RDD. This operations reads the file and treats every line as a separate object. We will use DisplayRDD() to visualize it. The second argument of `sc.textFile` is the number of partitions. We will set this as 10 to get started. If we don't do that, Spark will only create a single partition given the file is pretty small.

In [2]:
states_rdd = sc.textFile('states.txt', 10)
DisplayRDD(states_rdd)

AlabamaHawaiiMassachusettsNew MexicoSouth Dakota,AlaskaIdahoMichiganNew YorkTennesseeArizona,IllinoisMinnesotaNorth CarolinaTexas,ArkansasIndianaMississippiNorth DakotaUtah,CaliforniaIowaMissouriOhioVermontColorado,KansasMontanaOklahomaVirginiaConnecticutKentucky,NebraskaOregonWashingtonDelawareLouisiana,NevadaPennsylvaniaWest VirginiaFlorida,MaineNew HampshireRhode IslandWisconsinGeorgia,MarylandNew JerseySouth CarolinaWyoming


The above table shows the contents of each partition as a list -- so the first Partition has 5 elements in it ('Alabama', ...). We can `repartition` the RDD to get a fewer partitions so it will be easier to see.

In [3]:
states_rdd = states_rdd.repartition(5)
DisplayRDD(states_rdd)

NebraskaOregonWashingtonDelawareLouisianaMaineNew HampshireRhode IslandWisconsinGeorgia,AlabamaHawaiiMassachusettsNew MexicoSouth DakotaMarylandNew JerseySouth CarolinaWyoming,IllinoisMinnesotaNorth CarolinaTexasArkansasIndianaMississippiNorth DakotaUtahNevadaPennsylvaniaWest VirginiaFlorida,AlaskaIdahoMichiganNew YorkTennesseeArizona,CaliforniaIowaMissouriOhioVermontColoradoKansasMontanaOklahomaVirginiaConnecticutKentucky


Let's do a transformation where we convert a string to a 2-tuple, where the second value is the length of the string. We will just use a `map` for this -- we have to provide a function as the input that transforms each element of the RDD. In this case, we are using the `lambda` keyword to define a function inline.

The below lambda function is simply taking in a string: s, and returning a 2-tuple: (s, len(s))

In [4]:
states1 = states_rdd.map(lambda s: (s, len(s)))
DisplayRDD(states1)

"('Nebraska', 8)('Oregon', 6)('Washington', 10)('Delaware', 8)('Louisiana', 9)('Maine', 5)('New Hampshire', 13)('Rhode Island', 12)('Wisconsin', 9)('Georgia', 7)","('Alabama', 7)('Hawaii', 6)('Massachusetts', 13)('New Mexico', 10)('South Dakota', 12)('Maryland', 8)('New Jersey', 10)('South Carolina', 14)('Wyoming', 7)","('Illinois', 8)('Minnesota', 9)('North Carolina', 14)('Texas', 5)('Arkansas', 8)('Indiana', 7)('Mississippi', 11)('North Dakota', 12)('Utah', 4)('Nevada', 6)('Pennsylvania', 12)('West Virginia', 13)('Florida', 7)","('Alaska', 6)('Idaho', 5)('Michigan', 8)('New York', 8)('Tennessee', 9)('Arizona', 7)","('California', 10)('Iowa', 4)('Missouri', 8)('Ohio', 4)('Vermont', 7)('Colorado', 8)('Kansas', 6)('Montana', 7)('Oklahoma', 8)('Virginia', 8)('Connecticut', 11)('Kentucky', 8)"


Lets collect all the names with the same length together using a group by operation. 
```
groupByKey([numTasks])
```
When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable<V>) pairs. 
This wouldn't work as is, because `states1` is using the name as the key. Let's change that around.

In [5]:
states2 = states1.map(lambda t: (t[1], t[0]))
DisplayRDD(states2)

"(8, 'Nebraska')(6, 'Oregon')(10, 'Washington')(8, 'Delaware')(9, 'Louisiana')(5, 'Maine')(13, 'New Hampshire')(12, 'Rhode Island')(9, 'Wisconsin')(7, 'Georgia')","(7, 'Alabama')(6, 'Hawaii')(13, 'Massachusetts')(10, 'New Mexico')(12, 'South Dakota')(8, 'Maryland')(10, 'New Jersey')(14, 'South Carolina')(7, 'Wyoming')","(8, 'Illinois')(9, 'Minnesota')(14, 'North Carolina')(5, 'Texas')(8, 'Arkansas')(7, 'Indiana')(11, 'Mississippi')(12, 'North Dakota')(4, 'Utah')(6, 'Nevada')(12, 'Pennsylvania')(13, 'West Virginia')(7, 'Florida')","(6, 'Alaska')(5, 'Idaho')(8, 'Michigan')(8, 'New York')(9, 'Tennessee')(7, 'Arizona')","(10, 'California')(4, 'Iowa')(8, 'Missouri')(4, 'Ohio')(7, 'Vermont')(8, 'Colorado')(6, 'Kansas')(7, 'Montana')(8, 'Oklahoma')(8, 'Virginia')(11, 'Connecticut')(8, 'Kentucky')"


Note above that Spark did not do a shuffle to ensure that the same `keys` end up on the same partition. In fact, the `map` operation does not do a shuffle. 

Now we can do a groupByKey. 

In [6]:
states3 = states2.groupByKey()
DisplayRDD(states3)

"(10, )(5, )","(6, )(11, )","(12, )(7, )","(8, )(13, )","(9, )(14, )(4, )"


That looks weird... it seems to have done a group by, but we are missing the groups themselves. This is because the type of the value is a `pyspark.resultiterable.ResultIterable` which our DisplayRDD code does not translate into strings. We can fix that by converting the `values` to lists, and then doing DisplayRDD.

In [7]:
DisplayRDD(states3.mapValues(list))

"(10, ['Washington', 'New Mexico', 'New Jersey', 'California'])(5, ['Maine', 'Texas', 'Idaho'])","(6, ['Oregon', 'Hawaii', 'Nevada', 'Alaska', 'Kansas'])(11, ['Mississippi', 'Connecticut'])","(12, ['Rhode Island', 'South Dakota', 'North Dakota', 'Pennsylvania'])(7, ['Georgia', 'Alabama', 'Wyoming', 'Indiana', 'Florida', 'Arizona', 'Vermont', 'Montana'])","(8, ['Nebraska', 'Delaware', 'Maryland', 'Illinois', 'Arkansas', 'Michigan', 'New York', 'Missouri', 'Colorado', 'Oklahoma', 'Virginia', 'Kentucky'])(13, ['New Hampshire', 'Massachusetts', 'West Virginia'])","(9, ['Louisiana', 'Wisconsin', 'Minnesota', 'Tennessee'])(14, ['South Carolina', 'North Carolina'])(4, ['Utah', 'Iowa', 'Ohio'])"


There it goes. Now we can see that the operation properly grouped together the state names by their lengths. This operation required a `shuffle` since originally all names with length, say 10, were all over the place.

`groupByKey` does not reduce the size of the RDD. If we were interested in `counting` the number of states with a given length (i.e., a `group by count` query), we can use `reduceByKey` instead. However that requires us to do a map first.

In [8]:
states4 = states2.mapValues(lambda x: 1)
DisplayRDD(states4)

"(8, 1)(6, 1)(10, 1)(8, 1)(9, 1)(5, 1)(13, 1)(12, 1)(9, 1)(7, 1)","(7, 1)(6, 1)(13, 1)(10, 1)(12, 1)(8, 1)(10, 1)(14, 1)(7, 1)","(8, 1)(9, 1)(14, 1)(5, 1)(8, 1)(7, 1)(11, 1)(12, 1)(4, 1)(6, 1)(12, 1)(13, 1)(7, 1)","(6, 1)(5, 1)(8, 1)(8, 1)(9, 1)(7, 1)","(10, 1)(4, 1)(8, 1)(4, 1)(7, 1)(8, 1)(6, 1)(7, 1)(8, 1)(8, 1)(11, 1)(8, 1)"


`reduceByKey` takes in a single reduce function as the input which tells us what to do with any two values. In this case, we are simply going to use sum them up.

In [9]:
DisplayRDD(states4.reduceByKey(lambda v1, v2: v1 + v2))

"(10, 4)(5, 3)","(6, 5)(11, 2)","(12, 4)(7, 8)","(8, 12)(13, 3)","(9, 4)(14, 2)(4, 3)"


These operations could be done faster through using `aggregateByKey`, but the syntax takes some getting used to. `aggregateByKey` takes a `start` value, a function that tells it what to do for a given element in the RDD, and another reduce function. 

In [10]:
DisplayRDD(
    states2.aggregateByKey(0, lambda k, v: k + 1, lambda v1, v2: v1 + v2))

"(10, 4)(5, 3)","(6, 5)(11, 2)","(12, 4)(7, 8)","(8, 12)(13, 3)","(9, 4)(14, 2)(4, 3)"


### Basics 2: FlatMap

Unlike a `map`, the function used for `flatMap` returns a list -- this is used to allow for the possibility that we will generate different numbers of outputs for different elements. Here is an example where we split each string in `states_rdd` into multiple substrings.

The lambda function below splits a string into chunks of size 5: so 'South Dakota' gets split into 'South', ' Dako', 'ta', and so on. The lambda function itself returns a list. If you try this with 'map' the result would not be the same.

In [11]:
DisplayRDD(
    states_rdd.flatMap(
        lambda x: [str(x[i:i + 5]) for i in range(0, len(x), 5)]))

NebraskaOregonWashingtonDelawareLouisianaMaineNew HampshireRhode IslandWisconsinGeorgia,AlabamaHawaiiMassachusettsNew MexicoSouth DakotaMarylandNew JerseySouth CarolinaWyoming,IllinoisMinnesotaNorth CarolinaTexasArkansasIndianaMississippiNorth DakotaUtahNevadaPennsylvaniaWest VirginiaFlorida,AlaskaIdahoMichiganNew YorkTennesseeArizona,CaliforniaIowaMissouriOhioVermontColoradoKansasMontanaOklahomaVirginiaConnecticutKentucky


### Basics 3: Joins

Finally, lets look at an example of joins. We will still use small RDDs, but we now need two of them. We will just use `sc.parallelize` to create those RDDs. That functions takes in a list and creates an RDD of that by creating partitions and splitting them across machines. It takes the number of partitions as the second argument (optional).

Note again that Spark made no attempt to co-locate the objects (i.e., the tuples) with the same key.

In [12]:
rdd1 = sc.parallelize([('alpha', 1), ('beta', 2), ('gamma', 3), ('alpha', 5), ('beta', 6)], 3)
DisplayRDD(rdd1)

"('alpha', 1)","('beta', 2)('gamma', 3)","('alpha', 5)('beta', 6)"


In [13]:
rdd2 = sc.parallelize([('alpha', 'South Dakota'), ('beta', 'North Dakota'), ('zeta', 'Maryland'), ('beta', 'Washington')], 3)
DisplayRDD(rdd2)

"('alpha', 'South Dakota')","('beta', 'North Dakota')","('zeta', 'Maryland')('beta', 'Washington')"


Here is the definition of join from the programming guide.
```
When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key. Outer joins are supported through leftOuterJoin, rightOuterJoin, and fullOuterJoin. 
```
We want to join on the first attributes, so we can just call join directly, otherwise a map may have been required.

In [14]:
rdd3 = rdd1.join(rdd2)
DisplayRDD(rdd3)

,,,,,"('alpha', (1, 'South Dakota'))('alpha', (5, 'South Dakota'))('beta', (2, 'North Dakota'))('beta', (2, 'Washington'))('beta', (6, 'North Dakota'))('beta', (6, 'Washington'))"


There is a bunch of empty partitions. We could have controlled the number of partitions with an optional argument to join. But in any case, the output looks like what we were trying to do. Using `outerjoins` behaves as you would expect, with two extra tuples for fullOuterJoin.

In [15]:
DisplayRDD(rdd1.fullOuterJoin(rdd2))

,"('zeta', (None, 'Maryland'))",,"('gamma', (3, None))",,"('alpha', (1, 'South Dakota'))('alpha', (5, 'South Dakota'))('beta', (2, 'North Dakota'))('beta', (2, 'Washington'))('beta', (6, 'North Dakota'))('beta', (6, 'Washington'))"


`cogroup` is a related function, but basically creates two lists with each key. The `value` in that case is more complex, and our code above can't handle it. As we can see, there is a single object corresponding to each key, and the values are basically a pair of `iterables`.

In [16]:
DisplayRDD(rdd1.cogroup(rdd2).mapValues(lambda x: (list(x[0]), list(x[1]))))

,"('zeta', ([], ['Maryland']))",,"('gamma', ([3], []))",,"('alpha', ([1, 5], ['South Dakota']))('beta', ([2, 6], ['North Dakota', 'Washington']))"


### Basics 4: Word count

Here we will run some of the commands from the README file. This uses an RDD created from the lines of README.md file. You can use the DisplayRDD function here, but the output is rather large.

In [17]:
textFile = sc.textFile("README.md", 10)

In [18]:
textFile.count()

140

In [19]:
textFile.take(5)

['# Assignment 3: Apache Spark',
 '',
 "The goal of this assignment is to learn how to do large-scale data analysis tasks using Apache Spark: for this assignment, we will use relatively small datasets and  we won't run anything in distributed mode; however Spark can be easily used to run the same programs on much larger datasets.",
 '',
 '### Getting Started with Spark']

As described in the README file, the following command does a word count, by first separating out the words using a `flatMap`, and then using a `reduceByKey`.

In [20]:
counts = textFile.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
DisplayRDD(counts)

"('Assignment', 2)('', 78)('The', 16)('of', 40)('is', 25)('run', 4)('anything', 1)('basically', 3)('at', 6)('[Apache', 1)('(originally', 1)('instead', 1)('an', 12)('output', 7)('HDFS', 1)('1.', 2)('**Version', 1)('ready', 1)('languages:', 1)('quick', 2)('have', 2)('PYSPARK_DRIVER_PYTHON_OPTS=""notebook', 1)('--no-browser', 1)('Shell', 2)('relevant', 1)('python', 3)('sc.textFile(""README.md"")`:', 1)('commands', 3)('line', 6)('RDD).', 1)('(http://spark.apache.org/docs/latest/quick-start.html).', 1)('following', 3)('appears', 1)('Use', 1)('class,', 1)('preferable,', 1)('especially', 1)('mode.', 1)('look', 2)('Programming', 1)('Details', 1)('initializes', 1)('*', 4)('consisting', 4)('documents', 1)('defined', 1)('would', 6)('input', 5)('tuple', 2)('initial', 1)('""user1', 1)('was', 1)('Easiest', 1)('characters.', 1)('count', 1)('dictionaries).', 1)('starting', 1)('category', 1)('(just', 1)('format', 2)('Try', 1)('nodes', 3)('graph', 2)('Sample', 1)('results.txt', 1)('results', 1)","('how', 2)('analysis', 2)('and', 30)('much', 1)('computing', 1)('paradigm', 1)('by', 10)('Installing', 1)('you', 12)('download', 1)('Download', 1)('2.', 2)('available', 1)('`tar', 1)('here', 3)('follow', 2)('standard', 2)('Jupyter', 2)('Notebook', 3)('within', 1)('directly.', 1)('`$SPARKHOME/bin/pyspark`:', 1)('local', 2)('`sc.textFile`', 1)('(which', 3)('simply', 1)('application.', 1)('b)`', 1)('functions.', 1)('return', 3)('sum(a,', 1)('`flatmap`', 1)('splits', 1)('detailed', 1)('Walk-Through).', 1)('functions,', 2)('separate', 1)('write', 3)('which', 1)('program', 1)('RDDs:', 1)('user-product', 1)('Dataset', 1)('`functions.py`', 1)('(several', 1)('-', 10)('**Task', 10)('Write', 4)('lines)', 1)('product1', 1)('requires', 1)('average', 1)('`reduceByKey`', 1)('most', 1)('rating.', 1)('`(1995,', 1)('done', 1)('sanitized', 1)('any', 1)('selecting', 1)('>10).', 1)('parsed', 2)('document', 1)('surnames', 1)('Nobel', 2)('surnames.', 1)('JSONs', 2)('\ttask7_result', 1)('were', 1)('logs', 1)(""'01/Jul/1995'"", 1)('self-explanatory,', 1)('need:', 1)","('#', 1)('Spark', 14)('this', 6)('we', 5)('use', 12)('datasets', 2)('in', 20)('Started', 1)('new', 3)('Google', 1)('based', 1)('created', 1)('these', 2)('3.2.0,', 1)('downloaded', 2)('directory', 2)('3.', 2)('supports', 1)('way', 3)('\tPYSPARK_PYTHON=/usr/bin/python3', 1)('8881', 1)('file.', 2)('array', 1)('rest', 1)('Count', 2)('command', 2)('word:', 1)('textfile.flatMap(split).map(generateone).reduceByKey(sum)', 1)('(we', 1)('representation', 1)('compact', 1)('out', 4)('wordcount.py`', 1)('Shakespeare', 1)('examples', 1)('exception', 1)('5.0""', 1)('reviewed.', 1)('form', 2)('requests', 1)('teh', 1)('extract', 1)('LEONATO,', 1)('returns', 4)('already', 1)('transformations', 1)('print', 1)('*dates*', 1)('dates', 1)('[NASA', 1)('`amazonBipartiteRDD`).', 1)('aggregateByKey)', 1)('degree.', 2)('`spark-submit`', 1)('over).', 1)","('data', 2)('relatively', 2)('small', 3)('be', 17)('found', 1)('Spark](https://spark.apache.org)', 1)('cluster', 1)('2-stage', 1)('system);', 1)('items,', 1)('variety', 1)('or', 5)('RDDs', 2)('website.', 1)('package', 1)('it', 13)('(modify', 1)('Scala', 2)('(Spark', 1)('--', 2)('To', 2)('\t```', 2)('--port=8881""', 1)('also', 6)('about', 2)('doing).', 1)('initialized', 1)('called', 1)('`textFile`,', 1)('reading', 1)('output.', 1)('without', 1)('cluster.', 1)('runs', 1)('`$SPARKHOME/bin/spark-submit', 2)('ratings', 1)('(`amazon-ratings.txt`)', 1)('Noble', 1)('few', 1)('RDDs.', 1)('removing', 1)('first', 4)('Complete', 3)('computes', 3)('rating', 3)('2.87)`', 1)('(not', 2)('followed', 4)('So', 2)('outputs', 1)('In', 2)('parse', 1)('(this', 1)('Make', 2)('so', 1)('PairRDD', 2)('not', 1)('strings,', 1)('entries', 1)('easier', 1)('develop', 1)('(by', 1)","('easily', 1)('###', 9)('excellent', 2)('tutorials', 1)('[Spark', 2)('website](http://spark.apache.org).', 1)('developed', 1)('open-source', 1)('provides', 1)('set', 1)('more', 3)('RDD,', 2)('are', 10)('Since', 1)('large,', 1)('ask', 1)('uncomp